In [89]:
from qiskit import *
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver
from scipy.optimize import minimize
from qiskit.providers.aer import StatevectorSimulator

import numpy as np

import matplotlib.pyplot as plt

from numpy.random import random

In [90]:
import warnings
warnings.filterwarnings('ignore')

In [91]:
def create_hamiltonian(i,x,y,z):
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": i}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": z}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": x}, "label": "X"},
                   {"coeff": {"imag": 0.0, "real": y}, "label": "Y"}
                  ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [92]:
def create_pauli_operators(i,x,y,z):
    pauli_dict = { "X" : x,
                  "Y" : y,
                  "Z" : z,
                  "I" : i     
    }
    return pauli_dict

In [93]:
def create_ansatz(parameters):
    assert len(parameters) == 2
    circuit = QuantumCircuit(1,1)
    circuit.ry(parameters[0], 0)
    circuit.rx(parameters[1], 0)
    return circuit

In [94]:
def measure_operators(parameters, operator):
    circuit = create_ansatz(parameters)
    if operator == "I":
        return 1
    elif operator == "Z":
        circuit.measure(0, 0)
    elif operator == "Y":
        circuit.rx(np.pi / 2, 0)
        circuit.measure(0, 0)
    elif operator == "X":
        circuit.ry(np.pi / 2, 0)
        circuit.measure(0, 0)
    else:
        raise ValueError("Operator must be a valid Pauli operator")
    
#     display(circuit.draw(output='mpl'))
        
    # Measure value of cicuit
    num_shots = 10000
    qasm_sim = Aer.get_backend('qasm_simulator')
#     qasm_sim = StatevectorSimulator()
#     qobj = assemble(circuit, shots=num_shots, memory=True)
#     result = qasm_sim.run(qobj).result()
    job = execute(circuit, qasm_sim, shots=num_shots)
    result = job.result()
    counts = result.get_counts()
    total = 0
    for measurement in counts:
        sign = 1
        if measurement == '1':
            sign = -1
        total += sign * counts[measurement] / num_shots
    return total
        

In [95]:
def vqe(parameters):
    i_pauli = measure_operators(parameters, "I")
    z_pauli = measure_operators(parameters, "Z")
    y_pauli = measure_operators(parameters, "Y")
    x_pauli = measure_operators(parameters, "X")
    expectation_value = pauli_dict["I"] * i_pauli + pauli_dict["Z"] * z_pauli + pauli_dict["Y"] \
                        * y_pauli + pauli_dict["X"] * x_pauli
    return expectation_value

In [96]:
factor = 5
i,x,y,z = factor * random(), factor * random(), factor * random(), factor * random()

In [97]:
pauli_dict = create_pauli_operators(i, x, y, z)
array = [np.pi, np.pi]
minimum = minimize(vqe, array, method="Powell", tol=1e-3)
print("Estimated minimum eigenvalue: ", minimum.fun)

Estimated minimum eigenvalue:  -4.87454954604606


In [98]:
hamiltonian = create_hamiltonian(i, x, y, z)
exact_result = NumPyEigensolver(hamiltonian).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print("Exact ground state energy: ", reference_energy)

Exact ground state energy:  -4.841413018578932
